In [19]:
#importing modules
import pandas as pd
from pathlib import Path

In [20]:
#setting file path to read
voc_path = Path("Resources/voc_rehab.csv")
unemployment_path = Path("Resources/unemployment.csv")

# Read with Pandas
vocation_df = pd.read_csv(voc_path)
vocation_df.head()

,Year,County,Sex,Number of Consumers,Annotation Code,Count Annotation Desc,Percent,Percent Annotation Desc
0,2014,Alameda,Not Reported,4,NaN,NaN,NaN,NaN
1,2014,Alameda,Female,"2,356",NaN,NaN,NaN,NaN
2,2014,Alameda,Male,"2,391",NaN,NaN,NaN,NaN
3,2014,Alpine,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...
4,2014,Amador,Female,NaN,1.0,Cell suppressed for small number. A score over...,1.0,This value represents less than or equal to 1 ...


In [21]:
#shows what columns to determine relevance
vocation_df.columns

Index(['Year', 'County', 'Sex', 'Number of Consumers', 'Annotation Code',
       'Count Annotation Desc', 'Percent', 'Percent Annotation Desc'],
      dtype='object')

In [22]:
#removing irrelevant columns
del vocation_df["Annotation Code"]
del vocation_df["Count Annotation Desc"]
del vocation_df["Percent"]
del vocation_df["Percent Annotation Desc"]
del vocation_df["Sex"]
vocation_df.head()

,Year,County,Number of Consumers
0,2014,Alameda,4
1,2014,Alameda,"2,356"
2,2014,Alameda,"2,391"
3,2014,Alpine,NaN
4,2014,Amador,NaN


In [23]:
#filling na values w/0
voc_clean_df = vocation_df.fillna(0)

voc_clean_df.head()

,Year,County,Number of Consumers
0,2014,Alameda,4
1,2014,Alameda,"2,356"
2,2014,Alameda,"2,391"
3,2014,Alpine,0
4,2014,Amador,0


In [24]:
#organizing data by county
voc_clean_df.groupby("County").value_counts()

County   Year  Number of Consumers
Alameda  2014  2,356                  1
         2017  2,145                  1
               2                      1
               1,879                  1
         2016  5                      1
                                     ..
Yuba     2015  76                     1
               108                    1
         2014  58                     1
               108                    1
         2017  86                     1
Length: 553, dtype: int64

In [38]:
unemployment_df = pd.read_csv(unemployment_path)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
unemployment_df = pd.read_csv(unemployment_path)
unemployment_df = unemployment_df.set_index('Area Type')
unemployment_df

# Filter Area Type to only include County and State data and reduce columns
area_type_df = unemployment_df.loc[['State','County'],
                                   ['Area Name', 'Year', 'Labor Force', 'Employment', 'Unemployment', 'Unemployment Rate']]
area_type_df

# Group by Year and calculate averages
year_grouped = area_type_df.groupby(['Year', 'Area Name']).mean()
year_grouped

# Reduce to only 2017-2022
recent = year_grouped.loc[[2017, 2018, 2019, 2020, 2021, 2022]]
recent

Labor Force     Employment  Unemployment  \
Year Area Name                                                      
2017 Alameda County    838183.333333  807108.333333  31075.000000   
     Alpine County        547.500000     515.000000     35.833333   
     Amador County      14693.333333   13958.333333    735.000000   
     Butte County      102000.000000   96050.000000   5941.666667   
     Calaveras County   21016.666667   20013.333333   1000.833333   
...                              ...            ...           ...   
2022 Tulare County     207483.333333  190541.666667  16933.333333   
     Tuolumne County    19877.500000   18989.166667    888.333333   
     Ventura County    413641.666667  398416.666667  15216.666667   
     Yolo County       108991.666667  104575.000000   4416.666667   
     Yuba County        31583.333333   29841.666667   1733.333333   

                       Unemployment Rate  
Year Area Name                            
2017 Alameda County             0.037000  
     Alpine County              0.062917  
     Amador County              0.050167  
     Butte County               0.058250  
     Calaveras County           0.047583  
...                                  ...  
2022 Tulare County              0.081750  
     Tuolumne County            0.044667  
     Ventura County             0.037000  
     Yolo County                0.040417  
     Yuba County                0.055167  

[354 rows x 4 columns]